In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
os.environ['USE_PYGEOS'] = '0'
import pysal.lib
import helpers as hs
from importlib import reload
import folium
import pandas as pd
import geopandas as gpd

2. Load the filosofi file and build the dataframe

In [ ]:
cd ../data

In [ ]:
import py7zr
with py7zr.SevenZipFile('Filosofi2015_carreaux_200m_gpkg.7z', mode='r') as z:
    z.extractall()

In [ ]:
gdf = gpd.read_file("Filosofi2015_carreaux_200m_metropole.gpkg")

In [ ]:
gdfParis = gdf[gdf['Depcom'].str[:2].isin(["75"])]

In [ ]:
gdfParis.to_file("Filosofi2015_carreaux_200m_Paris.gpkg")

In [ ]:
filosofi_file = "Filosofi2015_carreaux_200m_Paris.gpkg" 
pgdf = hs.get_POI_cat_on_INSPIRE_grid(filosofi_file) 

In [ ]:
pgdf.to_file("pgdf.gpkg", layer="cool", driver="GPKG")

3. Create the r5 transport network

In [ ]:
transport_network = r5py.TransportNetwork("paris_expr.pbf", ["IDFM.gtfs.zip"])

Example: Visual presentation of number of restaurants in each square.

In [ ]:
hs.folium_color_map(pgdf, "restaurant")

In [ ]:
import datetime
import time

In [ ]:
pgdf_centered = pgdf.copy()
pgdf_centered['geometry'] = pgdf_centered['geometry'].apply(lambda x : x.centroid)

travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=pgdf_centered,
    destinations=pgdf_centered,
    departure=datetime.datetime(2024,10,8,11,59),
    transport_modes=[
        r5py.TransportMode.TRANSIT,
        r5py.TransportMode.WALK,
    ],
)

This computation takes around half an hour, for *just* Paris. Therefore, it can probably perform a similar computation for the Petite-Couronne region in reasonable time.

In [ ]:
travel_times = travel_time_matrix_computer.compute_travel_times()

In [ ]:
travel_times.to_csv("travel_times.csv")

To not run the computation again, results are given here:

In [ ]:
travel_times_vert = pd.read_csv("testtest.csv")

Auxilliary function changes them to the same format to adapt to what the previous work uses:

In [ ]:
travel_times_square = hs.vert_to_square(travel_times_vert)


**Important**: reading and writing to csv was done sometimes throughout the work, but it changes the dimensions of the dataframe by 1. Therefore, the function should adapt to accommodate both, but it doesn't yet. For running the code below, please use the imported .csv file.

*Note that further ahead there is another csv import, so this portion of the code can be safely skipped!*

In [ ]:
travel_times_square = pd.read_csv("travel_times_square.csv")

In [ ]:
travel_times_square = travel_times_square.set_index('id')

In [ ]:
weights_by_id = hs.transfer_time_to_weight_faster(travel_times_square)

In [ ]:
weights_by_id = pd.read_csv("weights_by_id.csv")
weights_by_id.columns
weights_by_id = weights_by_id.drop('Unnamed: 0', axis=1)
#weights_by_id = weights_by_id.set_index('id')

In [ ]:
weights_by_id.head()

## Example: weights from the square containing Chatelet Les-Halles station

In [ ]:
value = 'CRS3035RES200mN2890000E3760400'
losc = weights_by_id.where(weights_by_id == value).stack().index.tolist()

example = pgdf[["geometry", "id"]].assign(weight_0 = weights_by_id.iloc[losc[0][0],1:].values)
example

In [ ]:
hs.folium_color_map(example, "weight_0")

We can verify now that the results make sense: Observe that the outliers with higher weights correspond closely to metro stops. Moreover, the map above doesn't consider all the transit lines, but the results do, which explains some other outliers (Consider Alesia and Porte d'Orleans, which on the map have slightly higher weights than their surrounding area, which probably results from them being located close to metro stations, which are not shown on the map).